In [ ]:
import numpy as np
import pandas as pd
import requests
from urllib.parse import urlencode

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## List of Genres `id` and `name`

| Genre Name | ID   | Genre Name  | ID    | Genre Name      | ID  | Genre Name | ID    |
| ---------- | ---- | ----------- | ----- | --------------- | --- | ---------- | ----- |
| Action     | 28   | Adventure   | 12    | Animation       | 16  | Comedy     | 35    |
| Crime      | 80   | Documentary | 99    | Drama           | 18  | Family     | 10751 |
| Fantasy    | 14   | History     | 36    | Horror          | 27  | Music      | 10402 |
| Mystery    | 9648 | Romance     | 10749 | Science Fiction | 878 | TV Movie   | 10770 |
| Thriller   | 53   | War         | 10752 | Western         | 37  | —          | —     |
| Fantasy         | 14    | War             | 10752 |












- Action
- Romance
- Crime
- 

# Start Scrapping

In [47]:


import requests
import pandas as pd
from urllib.parse import urlencode
import time

api_key = "976b276ecf310bf8db66270ad372aecb"
base_url = "https://api.themoviedb.org/3/discover/movie"

date_ranges = [
    ("2011-01-01", "2012-12-31"),
    ("2013-01-01", "2014-12-31"),
    ("2015-01-01", "2016-12-31")
]

final = []

for start_date, end_date in date_ranges:
    params = {
        "api_key": api_key,
        "language": "en-US",
        "sort_by": "primary_release_date.desc",
        "include_adult": "true",
        "include_video": "false",
        "primary_release_date.gte": start_date,
        "primary_release_date.lte": end_date,
        "with_original_language": "en",
        "with_genres": "27",
        "without_genres": "16",   # Exclude animation
    }

    # Get total pages from first call
    first_url = f"{base_url}?{urlencode(params)}&page=1"
    first_response = requests.get(first_url).json()
    
    # extract total pages and movies
    pages_found = first_response.get("total_pages")
    movies_found = first_response.get("total_results")
    total_pages = min(first_response.get("total_pages", 1), 500)
    total_movies = total_pages*20
    print(f"Pages Found  : {pages_found}\nMovies Found : {movies_found}\n\nFetching movies:\n- From  : {start_date} \n- To    : {end_date} \n- Pages : {total_pages} \n- Movies: {total_movies}\n")
    
    # Loop through pages
    for page in range(1, total_pages + 1):
        url = f"{base_url}?{urlencode(params)}&page={page}"
        response = requests.get(url).json()
    
        # Check if "results" key exists
        if "results" not in response:
            print(f"⚠️ Skipping page {page} | Message: {response}")
            continue
    
        # Create DataFrame for each (1) page
        df = pd.DataFrame(response["results"])[[
            "id", "title", "overview", "release_date",
            "original_language", "genre_ids", "adult", "popularity"
        ]]
        final.append(df)
    
        # Print progress
        if page % 20 == 0:
            #print(f"✅ Page {page}/{total_pages} | Movies: {sum(len(d) for d in final)}")
    
            print(f"✅ Collectect Page: {page}/{total_pages} | Collected Movies: {sum(len(d) for d in final)}")

        #time.sleep(0.25)
    
# Combine all pages
final_df = pd.concat(final, ignore_index=True)
print("\nPlease wait a minute...")
print("\n✅ Done!")
print("Total Movies Collected:", len(final_df))
print("x==================x")
    
# Preview the data
print(final_df.head())

Pages Found  : 85
Movies Found : 1681

Fetching movies:
- From  : 2011-01-01 
- To    : 2012-12-31 
- Pages : 85 
- Movies: 1700

✅ Collectect Page: 20/85 | Collected Movies: 400
✅ Collectect Page: 40/85 | Collected Movies: 800
✅ Collectect Page: 60/85 | Collected Movies: 1200
✅ Collectect Page: 80/85 | Collected Movies: 1600
Pages Found  : 110
Movies Found : 2183

Fetching movies:
- From  : 2013-01-01 
- To    : 2014-12-31 
- Pages : 110 
- Movies: 2200

✅ Collectect Page: 20/110 | Collected Movies: 2081
✅ Collectect Page: 40/110 | Collected Movies: 2481
✅ Collectect Page: 60/110 | Collected Movies: 2881
✅ Collectect Page: 80/110 | Collected Movies: 3281
✅ Collectect Page: 100/110 | Collected Movies: 3680
Pages Found  : 121
Movies Found : 2409

Fetching movies:
- From  : 2015-01-01 
- To    : 2016-12-31 
- Pages : 121 
- Movies: 2420

✅ Collectect Page: 20/121 | Collected Movies: 4263
✅ Collectect Page: 40/121 | Collected Movies: 4663
✅ Collectect Page: 60/121 | Collected Movies: 5063

KeyboardInterrupt: 

In [29]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8101 entries, 0 to 8100
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 8101 non-null   int64  
 1   title              8101 non-null   object 
 2   overview           8101 non-null   object 
 3   release_date       8101 non-null   object 
 4   original_language  8101 non-null   object 
 5   genre_ids          8101 non-null   object 
 6   adult              8101 non-null   bool   
 7   popularity         8101 non-null   float64
dtypes: bool(1), float64(1), int64(1), object(5)
memory usage: 451.1+ KB


## Save the DataSet

In [ ]:
final.to_csv('test1.csv')

In [ ]:
final.to_csv('all_movies_(1980-2025).csv')

## Load the DataSet

In [ ]:
movies = pd.read_csv('test1.csv')
movies.info()
movies.shape

## Experiments

In [ ]:

# Print progress
    if page % 50 == 0 or page == total_pages:
        print(f"✅ Page {page}/{total_pages} | Movies: {sum(len(d) for d in final)}")

In [15]:
final.shape

(11300, 8)

## Check Duplicate Values

In [18]:
duplicates_by_key = final.duplicated(subset=['id'])
print("Duplicates by id:", duplicates_by_key.sum())


Duplicates by id: 11280
